In [8]:
# === Cell 1: Library Imports & Setup ===
import os
import gc
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
from scipy.stats import skew, kurtosis  # 新增這個！
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

# 設定隨機種子 (為了讓結果可重現，這對比賽很重要)
np.random.seed(42)

DATA_DIR = "/kaggle/input/data02" # 記得確認你的路徑是否正確

# 讀取 Log 檔
train_log = pd.read_csv(f"{DATA_DIR}/train_log.csv")
test_log  = pd.read_csv(f"{DATA_DIR}/test_log.csv")

print("Libraries loaded & Random Seed Fixed.")

Libraries loaded & Random Seed Fixed.


In [9]:
# === Cell 2: Advanced Feature Engineering ===

def calculate_eta(x):
    """
    計算 von Neumann Ratio (Eta)。
    數值越小 = 軌跡越平滑 (規律)；數值越大 = 越像雜訊。
    """
    if len(x) < 2:
        return 0.0
    sigma2 = np.var(x, ddof=1)
    if sigma2 == 0:
        return 0.0
    diff = np.diff(x)
    return np.mean(diff**2) / sigma2

def build_features_from_lightcurves(df):
    # 1. 預處理：確保按時間排序
    df = df.sort_values(["object_id", "Filter", "Time (MJD)"])
    
    # 建立加權平均用的權重 (Weight = 1 / error^2)
    df['w'] = 1.0 / (df['Flux_err']**2 + 1e-9)
    df['flux_w'] = df['Flux'] * df['w']

    # 2. 向量化聚合 (Aggregation)
    # 加入 skew (偏度) 和 kurtosis (峰度)
    aggs = {
        'Flux': ['min', 'max', 'mean', 'median', 'std', skew, kurtosis], 
        'Flux_err': ['mean'],
        'Time (MJD)': ['min', 'max', 'count'], 
        'w': ['sum'],
        'flux_w': ['sum']
    }
    
    # 執行 Groupby 計算
    agg_df = df.groupby(['object_id', 'Filter']).agg(aggs)
    agg_df.columns = [f"{k}_{v}" for k, v in agg_df.columns]
    agg_df = agg_df.reset_index()
    
    # 3. 計算聚合後的基礎特徵
    agg_df['flux_amp'] = agg_df['Flux_max'] - agg_df['Flux_min'] # 振幅
    agg_df['flux_rel_amp'] = agg_df['flux_amp'] / (agg_df['Flux_mean'] + 1e-9) # 相對振幅
    agg_df['time_span'] = agg_df['Time (MJD)_max'] - agg_df['Time (MJD)_min'] # 觀測時間跨度
    
    # 計算加權平均亮度 (比普通平均更準)
    agg_df['flux_w_mean'] = agg_df['flux_w_sum'] / (agg_df['w_sum'] + 1e-9)
    
    # 4. 計算變異性指標 (Eta) - 使用 apply
    # 這是天文學判斷變星的重要指標
    eta_df = df.groupby(['object_id', 'Filter'])['Flux'].apply(calculate_eta).reset_index()
    eta_df.columns = ['object_id', 'Filter', 'flux_eta']
    
    # 合併 Eta
    agg_df = agg_df.merge(eta_df, on=['object_id', 'Filter'], how='left')
    
    # 清理中間變數以節省記憶體
    agg_df.drop(columns=['w_sum', 'flux_w_sum'], inplace=True)

    # 5. Pivot (轉置): 將 Filter 0, 1... 轉成欄位
    agg_df['has_filter'] = 1
    
    # Filter 存在性特徵
    filter_presence = agg_df.pivot(index="object_id", columns="Filter", values="has_filter").fillna(0)
    filter_presence.columns = [f"has_{c}" for c in filter_presence.columns]
    
    # 數值特徵轉置
    numeric_cols = [c for c in agg_df.columns if c not in ['object_id', 'Filter', 'has_filter']]
    feats_wide = agg_df.pivot(index="object_id", columns="Filter", values=numeric_cols)
    feats_wide.columns = [f"{c[0]}_{c[1]}" for c in feats_wide.columns] # e.g., Flux_mean_0
    
    feats_wide = feats_wide.reset_index().fillna(0.0)
    filter_presence = filter_presence.reset_index()

    final_df = feats_wide.merge(filter_presence, on="object_id", how="left")
    
    # ==========================================
    # 6. 新增：Color Features (顏色特徵 - 物理核心)
    # ==========================================
    # 自動偵測有哪些濾鏡，並計算它們之間的亮度差 (顏色)
    available_filters = sorted(df['Filter'].unique())
    
    for i in range(len(available_filters)):
        for j in range(i + 1, len(available_filters)):
            f1 = available_filters[i]
            f2 = available_filters[j]
            
            # 使用 Weighted Mean 來算顏色
            col1 = f"flux_w_mean_{f1}"
            col2 = f"flux_w_mean_{f2}"
            
            if col1 in final_df.columns and col2 in final_df.columns:
                # 顏色 (亮度差) -> 代表溫度
                final_df[f"color_{f1}_{f2}"] = final_df[col1] - final_df[col2]
                # 顏色比例
                final_df[f"color_ratio_{f1}_{f2}"] = final_df[col1] / (final_df[col2] + 1e-6)

    return final_df

In [10]:
# === Cell 3: Data Loading & Physics Interactions ===

def load_and_process_splits(split_ids, mode="train"):
    all_feats = []
    
    for i in split_ids:
        fname = f"{DATA_DIR}/split_{i:02d}/{mode}_full_lightcurves.csv"
        if os.path.exists(fname):
            print(f"Processing {fname}...")
            df = pd.read_csv(fname)
            feats = build_features_from_lightcurves(df)
            all_feats.append(feats)
            
            # 記憶體管理
            del df
            gc.collect()
            
    full_df = pd.concat(all_feats, ignore_index=True)
    
    # 處理重複的 object_id (取平均)
    if full_df.duplicated(subset=['object_id']).any():
        full_df = full_df.groupby("object_id", as_index=False).mean()
        
    return full_df

# 讀取 Train (這一步會跑比較久，因為特徵變多了)
print("Loading TRAIN data with advanced features...")
train_feats = load_and_process_splits(range(1, 21), mode="train")

# 合併 Meta Data
train = train_feats.merge(
    train_log[["object_id","Z","Z_err","EBV","target"]],
    on="object_id",
    how="left"
).fillna(0)

# ==========================================
# Level 2 特徵工程：物理交互項
# ==========================================

# 1. 亮度與距離(Z)的交互作用 -> 模擬「絕對星等」
for col in train.columns:
    if "flux_w_mean" in col: # 針對所有濾鏡的平均亮度
        train[col + "_x_Z"] = train[col] * train["Z"]
        # train[col + "_x_Z_sq"] = train[col] * (train["Z"]**2) # 可選：距離平方反比

# 2. 顏色與 EBV (星際消光) 的修正
# EBV 會讓星星變紅，我們要扣掉這個影響
for col in train.columns:
    if "color_" in col and "ratio" not in col:
        train[col + "_minus_EBV"] = train[col] - train["EBV"]

print("Train shape:", train.shape)
print("Ready for training!")

Loading TRAIN data with advanced features...
Processing /kaggle/input/data02/split_01/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_02/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_03/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_04/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_05/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_06/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_07/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_08/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_09/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_10/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_11/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_12/train_full_lightcurves.csv...
Processing /kaggle/input/data02/split_13/train_full_lightcurves.csv...
Processing /kaggle/input/data02/

In [11]:
feature_cols = [c for c in train.columns if c not in ["object_id", "target"]]
X = train[feature_cols].values
y = train["target"].values

# 設定 K-Fold
N_FOLDS = 5
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

# 儲存預測結果
oof_preds = np.zeros(X.shape[0])
models_lgb = []
models_xgb = []

# 計算權重
pos = (y == 1).sum()
neg = (y == 0).sum()
scale_pos_weight = neg / (pos + 1)

# 參數設定
params_lgb = {
    "objective": "binary",
    "metric": "binary_logloss",
    "learning_rate": 0.03,
    "num_leaves": 64, # 稍微調小避免過擬合
    "feature_fraction": 0.7,
    "bagging_fraction": 0.7,
    "bagging_freq": 1,
    "scale_pos_weight": scale_pos_weight,
    "verbosity": -1,
}

params_xgb = {
    "max_depth": 6,
    "eta": 0.03,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "scale_pos_weight": scale_pos_weight,
    "tree_method": "hist" # 加速
}

print(f"Starting {N_FOLDS}-Fold Cross Validation...")

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    print(f"=== FOLD {fold+1}/{N_FOLDS} ===")
    
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]
    
    # --- LightGBM ---
    dtrain_lgb = lgb.Dataset(X_train, label=y_train)
    dval_lgb = lgb.Dataset(X_val, label=y_val)
    
    clf_lgb = lgb.train(
        params_lgb,
        dtrain_lgb,
        num_boost_round=1000,
        valid_sets=[dtrain_lgb, dval_lgb],
        callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)] # 靜默模式
    )
    models_lgb.append(clf_lgb)
    val_pred_lgb = clf_lgb.predict(X_val)
    
    # --- XGBoost ---
    dtrain_xgb = xgb.DMatrix(X_train, label=y_train)
    dval_xgb = xgb.DMatrix(X_val, label=y_val)
    
    clf_xgb = xgb.train(
        params_xgb,
        dtrain_xgb,
        num_boost_round=1000,
        evals=[(dval_xgb, "eval")],
        early_stopping_rounds=50,
        verbose_eval=False
    )
    models_xgb.append(clf_xgb)
    val_pred_xgb = clf_xgb.predict(dval_xgb)
    
    # Ensemble OOF
    oof_preds[val_idx] = (val_pred_lgb + val_pred_xgb) / 2

# 尋找最佳閾值 (基於 OOF)
best_thr, best_f1 = 0, 0
for thr in np.linspace(0.05, 0.95, 100):
    pred_label = (oof_preds >= thr).astype(int)
    f1 = f1_score(y, pred_label)
    if f1 > best_f1:
        best_f1 = f1
        best_thr = thr

print(f"\nCV F1 Score: {best_f1:.4f}")
print(f"Best Threshold: {best_thr:.3f}")

Starting 5-Fold Cross Validation...
=== FOLD 1/5 ===
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[70]	training's binary_logloss: 0.0436114	valid_1's binary_logloss: 0.157195
=== FOLD 2/5 ===
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[99]	training's binary_logloss: 0.0278837	valid_1's binary_logloss: 0.146123
=== FOLD 3/5 ===
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[52]	training's binary_logloss: 0.0561431	valid_1's binary_logloss: 0.152988
=== FOLD 4/5 ===
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[91]	training's binary_logloss: 0.0315416	valid_1's binary_logloss: 0.137688
=== FOLD 5/5 ===
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[106]	training's binary_logloss: 0.025914	valid_1's binary_logloss: 0.122066

CV F1 Score: 

In [12]:
# 讀取 Test (使用與 Train 相同的邏輯)
test_feats = load_and_process_splits(range(1, 21), mode="test")

test = test_feats.merge(
    test_log[["object_id","Z","Z_err","EBV"]],
    on="object_id",
    how="left"
).fillna(0)

# Test Z Interaction
interact_bases = ["Flux_mean", "Flux_std", "flux_amp", "flux_slope", "flux_w_mean"]
for base in interact_bases:
    cols = [c for c in test.columns if base in c]
    for col in cols:
        test[col + "_Z"] = test[col] * test["Z"]

# 確保欄位一致
X_test = test[feature_cols].values
dtest = xgb.DMatrix(X_test)

# 推論 (對所有 Fold 取平均)
sub_preds = np.zeros(X_test.shape[0])

print("Predicting on test set...")
for i in range(N_FOLDS):
    pred_lgb = models_lgb[i].predict(X_test)
    pred_xgb = models_xgb[i].predict(dtest)
    sub_preds += (pred_lgb + pred_xgb) / 2

sub_preds /= N_FOLDS
print("Prediction complete.")

Processing /kaggle/input/data02/split_01/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_02/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_03/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_04/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_05/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_06/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_07/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_08/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_09/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_10/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_11/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_12/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_13/test_full_lightcurves.csv...
Processing /kaggle/input/data02/split_14/test_full_lightcurves.csv...
Processing /kaggle/i

KeyError: "['flux_w_mean_g_x_Z', 'flux_w_mean_i_x_Z', 'flux_w_mean_r_x_Z', 'flux_w_mean_u_x_Z', 'flux_w_mean_y_x_Z', 'flux_w_mean_z_x_Z', 'color_g_i_minus_EBV', 'color_g_r_minus_EBV', 'color_g_u_minus_EBV', 'color_g_y_minus_EBV', 'color_g_z_minus_EBV', 'color_i_r_minus_EBV', 'color_i_u_minus_EBV', 'color_i_y_minus_EBV', 'color_i_z_minus_EBV', 'color_r_u_minus_EBV', 'color_r_y_minus_EBV', 'color_r_z_minus_EBV', 'color_u_y_minus_EBV', 'color_u_z_minus_EBV', 'color_y_z_minus_EBV'] not in index"

In [ ]:
# ===========================
# 修改版 Cell 6：手動測試更低的閾值
# ===========================

# 因為 0.365 分數變高，我們繼續往下降來找最高分
# 之前的最佳紀錄是 0.14 (舊模型)，新模型可能落在 0.2~0.3 之間
threshold_list = [
    0.20,
    0.24,
    0.28,
    0.32,
    0.35  # 接續剛剛的 0.365
]

print("Generating lower threshold submissions:", threshold_list)

for idx, thr in enumerate(threshold_list):
    # 使用記憶體中已經算好的 sub_preds
    pred_sub = (sub_preds >= thr).astype(int)
    
    sub_df = pd.DataFrame({
        "object_id": test["object_id"],
        "prediction": pred_sub
    })
    
    # 檔名加上 low_thr 標記以便區分
    fname = f"submission_low_thr{idx+1}_{thr:.2f}.csv"
    sub_df.to_csv(fname, index=False)
    
    print(f"Saved: {fname} (Positive count: {pred_sub.sum()})")

print("\nNew low-threshold files generated!")